In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import datetime
import PIL
import tensorflow as tf
import requests

from tensorflow import keras
from tensorflow.keras import layers, models, applications

# Note : we are using TensorFlow Core v2.5.0, in TensorFlow Core v2.6.0 all the data 
# augmentation layers are part of tf.keras.layers
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation, RandomZoom
from tensorflow.keras import Input, Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard, ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adamax

In [2]:
TRAIN_DIR = "../raw_data/wikiart/wikiart-target_style-class_14-keepgenre_True-merge_style_m1-flat_False/train"
VAL_DIR = "../raw_data/wikiart/wikiart-target_style-class_14-keepgenre_True-merge_style_m1-flat_False/val"
TEST_DIR = "../raw_data/wikiart/wikiart-target_style-class_14-keepgenre_True-merge_style_m1-flat_False/test"

BATCH_SIZE = 128 # Hyper param, you can tune it
EPOCHS = 1000 # Large number, early stopping to stop training before this number
IMG_HEIGHT = 224 # VGG's dim
IMG_WIDTH = 224 # VGG's dim
NUM_CLASSES = 14 # Number of art styles

In [3]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory=TRAIN_DIR,
    labels='inferred',
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    label_mode='categorical',
    shuffle=True)

assert len(train_ds.class_names) == NUM_CLASSES

Found 46971 files belonging to 14 classes.


In [4]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory=VAL_DIR,
    labels='inferred',
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    label_mode='categorical',
    shuffle=True)

assert len(val_ds.class_names) == NUM_CLASSES

Found 5871 files belonging to 14 classes.


In [5]:
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory=TEST_DIR,
    labels='inferred',
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    label_mode='categorical',
    shuffle=True)

assert len(test_ds.class_names) == NUM_CLASSES


Found 5872 files belonging to 14 classes.


In [6]:
total_images_count = (int(len(list(train_ds)))+int(len(list(val_ds)))+int(len(list(test_ds))))*BATCH_SIZE

In [7]:
total_images_count

58752

In [8]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

# Optimizing the dataset by caching and prefetching the data
train_ds = train_ds.cache().shuffle(int(total_images_count)).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [9]:
layer_model = applications.VGG16(
    include_top=False, # We do not include VGG classification layers
    weights='imagenet', # We import VGG pre-trained on ImageNet
    input_shape=(IMG_HEIGHT, IMG_WIDTH, 3), 
    classes=NUM_CLASSES)

In [10]:
# layer_model.trainable = False # We do not train VGG weights

# layer_model.layers[-2:] # Set the two last layers as trainable
# for layer in layer_model.layers[-4:]:
#     layer.trainable = True

trainable_layer_count = 0
for i in range(len(layer_model.layers)):
    if layer_model.layers[i].trainable:
        trainable_layer_count += 1

layer_model.trainable = True

trainable_layer_count


19

In [11]:
data_augmentation_layers = tf.keras.models.Sequential([
    RandomFlip("horizontal", input_shape=(224, 224, 3)),
    RandomRotation(0.3),
    RandomZoom(0.3)
])

In [12]:
tf.keras.backend.clear_session()

inputs = Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))

x = data_augmentation_layers(inputs)

x = applications.vgg16.preprocess_input(x)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.5)(x)

outputs = layers.Dense(NUM_CLASSES, activation='softmax', name="classification_layer")(x)

model = Model(inputs, outputs)


In [13]:
es = EarlyStopping(monitor='val_loss', patience=10, mode='min', restore_best_weights=True)

# You can add it to the callbacks if you want to save checkpoints
checkpoint_dir = "../VGG16/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + f"-unfreeze_{trainable_layer_count}"
mcp = ModelCheckpoint(
    filepath=checkpoint_dir,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_freq=10,
    save_best_only=True)


In [14]:
%load_ext tensorboard
recorded_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S") 
log_dir = f"../VGG16/" + \
    recorded_time + \
    f"-images_{total_images_count}" + \
    f"-unfreeze_{trainable_layer_count}" + \
    f"-batch_{BATCH_SIZE}"

tsboard = TensorBoard(log_dir=log_dir)

In [15]:
model.compile(optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001), 
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), 
              metrics=['accuracy'])

history = model.fit(
    train_ds, 
    epochs=EPOCHS, 
    validation_data=val_ds, 
    callbacks=[es, tsboard], 
    use_multiprocessing=True)

Epoch 1/1000


/Users/jiangtianhan/Desktop/You will get good one day/TheUltimateAi/notebook/.venv/lib/python3.11/site-packages/keras/src/backend.py:5562: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(
2023-10-11 10:39:38.280500: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] Filling up shuffle buffer (this may take a while): 12 of 58752
2023-10-11 10:39:48.548006: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] Filling up shuffle buffer (this may take a while): 32 of 58752
2023-10-11 10:39:58.434776: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] Filling up shuffle buffer (this may take a while): 52 of 58752
2023-10-11 10:40:08.177018: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] Filling up shuffle buffer (this may take a while): 71 of 58752
2023-10-11 10:40:18.343187: I tensorflow

367/367 [==============================] - 288s 273ms/step - loss: 37.4860 - accuracy: 0.1015 - val_loss: 18.7871 - val_accuracy: 0.0330
Epoch 2/1000
367/367 [==============================] - 88s 238ms/step - loss: 21.2460 - accuracy: 0.1359 - val_loss: 7.6009 - val_accuracy: 0.1042
Epoch 3/1000
367/367 [==============================] - 86s 235ms/step - loss: 9.6869 - accuracy: 0.1655 - val_loss: 3.2438 - val_accuracy: 0.2076
Epoch 4/1000
367/367 [==============================] - 92s 251ms/step - loss: 3.0515 - accuracy: 0.2558 - val_loss: 2.2449 - val_accuracy: 0.3018
Epoch 5/1000
367/367 [==============================] - 93s 252ms/step - loss: 2.2211 - accuracy: 0.2949 - val_loss: 2.1897 - val_accuracy: 0.2843
Epoch 6/1000
367/367 [==============================] - 100s 273ms/step - loss: 2.1785 - accuracy: 0.2926 - val_loss: 2.1562 - val_accuracy: 0.2916
Epoch 7/1000
367/367 [==============================] - 101s 275ms/step - loss: 2.1524 - accuracy: 0.2927 - val_loss: 2.1372 -

In [19]:
model.evaluate(test_ds, callbacks=tsboard)

46/46 [==============================] - 24s 417ms/step - loss: 2.1076 - accuracy: 0.2835


[2.107649564743042, 0.283549040555954]

In [20]:
model.save('model.h5')

/Users/jiangtianhan/Desktop/You will get good one day/TheUltimateAi/notebook/.venv/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


: 